<h1> JAY BAKSHI : 50206954 </h1>
<br/> This notebook follows the  dplyr vignette available @ https://cran.r-project.org/web/packages/dplyr/vignettes/databases.html

Installing required packages

In [ ]:
install.packages('dpylr', repos = 'https://cran.r-project.org')

In [ ]:
install.packages('nycflights13', repos = 'https://cran.r-project.org')

In [ ]:
install.packages('RPostgreSQL', repos= 'http://cran.r-project.org')

In [ ]:
install.packages('DBI', repos="https://cran.r-project.org")

In [ ]:
install.packages('sqldf', repos="https://cran.r-project.org")

In [ ]:
library(dplyr)

In [ ]:
library(sqldf)

In [ ]:
library(DBI)

In [ ]:
library(RPostgreSQL)

Establishing a SQLite Db and connection with it

In [ ]:
my_db <- src_sqlite("my_db.sqlite3", create=T)

<b><code>my_db</code></b> has no data in it @ moment, so importing the <b><code>flights</code></b> data using <b><code>copy_to()</code></b> fn
<br/><mark> This puts all the data in the memory which is not efficient and not practically possible for large datasets. </mark>

In [ ]:
library(nycflights13)

Transferring the data into the SQLite Db

In [ ]:
flights_sqlite <- copy_to(my_db, flights, temporary = FALSE, indexes = list(c("year", "month", "day"), "carrier", "tailnum"))

built-in <b><code>src</code></b> will cache <b><code>flights</code></b> in a standard location

In [ ]:
flights_sqlite <- tbl(nycflights13_sqlite(), "flights")

In [ ]:
flights_sqlite

In [ ]:
class(flights_sqlite)

The SQL command for the same

In [ ]:
tbl(my_db, sql("SELECT * FROM flights"))

<h3> Basic Verbs </h3>

In [ ]:
select(flights_sqlite, year:day, dep_delay, arr_delay)

In [ ]:
filter(flights_sqlite, dep_delay > 240)

In [ ]:
arrange(flights_sqlite, year, month, day)

In [ ]:
mutate(flights_sqlite, speed = air_time / distance)

In [ ]:
summarise(flights_sqlite, delay = mean(dep_time))

The expressions in <b><code> select(), filter(), arrange(), mutate() and summarize() </code></b> are translated into SQL to run on Db, but not all the translations are perfect and limitations are present.

<h3> Laziness </h3>
<br/> dplyr -
1. Pulls data into R only when explicitly asked to
2. Delays and collects everything to do on db and communicate it all in one step

In [ ]:
c1 <- filter(flights_sqlite, year == 2013, month ==1, day ==1)

In [ ]:
c2 <- select(c1, year, month, day, carrier, dep_delay, air_time, distance)

In [ ]:
c3 <- mutate(c2, speed = distance / air_time * 60)

In [ ]:
c4 <- arrange(c3, year, month, day, carrier)

The above sequenec is not even sent to db, till explicity we ask for the data e.g. printing them.

In [ ]:
c1

In [ ]:
c2

In [ ]:
c3

In [ ]:
c4

<mark>To pull down all the results using <b><code> collect() </code></b>, which returns a <b><code> tbl_df() </code></b>

In [ ]:
collect(c4)

Query <b><code> dplyr </code></b> has generated by looking at the <b><code> query </code></b> component of the object:

In [ ]:
c4$query

<b><code> explain() </code></b> will ask the Db how it plans to execute the query, just keep an idea of the indexes that is being used for the dataset that we're using to understand the ouput better.

In [ ]:
explain(c4)

<h3> Forcing computation </h3>
1. <b><code> collect() </code></b> executes query to return results to R in memory.
2. <b><code> compute() </code></b> executes query to store results in temporary table in Db.
3. <b><code> collapse() </code></b> turns query to a Table expression.

<br/><h3> Performance considerations</h3>
<b><code> dplyr </code></b> prevents execution of expensive query operations. The ways it does is this -
1. <b><code> nrow() </code></b> returns <b><code> NA </code></b> so there is no way to know how many rows will return till actually execute the query
2. <b><code> tbl </code></b> will run the query for the first 10 rows only.
3. Usage of <b><code> tail() </code></b> is not possible till query is ran throughout.

<h3> SQL Translation examples </h3>

In [ ]:
translate_sql(x)

In [ ]:
translate_sql("x")

In [ ]:
translate_sql(x == 1 && (y < 2 || z > 3))

In [ ]:
translate_sql(x ^ 2 < 10)

In [ ]:
translate_sql(x %% 2 == 10)

In [ ]:
translate_sql(1)

In [ ]:
translate_sql(1L)

The goal of <b><code> dplyr </code></b> is to provide a semantic and not a literal translation of SQL > R.
<br/> R has many more functions that Db have. So much so that functions common in both can leave us different outputs.

In [ ]:
translate_sql(mean(x, trim = T))

<mark>Any fn that <b><code> dplyr </code></b> doesn't know how to convert is left as is.</mark>

In [ ]:
translate_sql(glob(x, y))

In [ ]:
translate_sql(x %like% "ab*")

<h3> Grouping </h3>
<br/> The only really useful operations for grouped SQLite tables are found in <b><code> summarize() </code></b>.
<br/> Db always drops NULLs so we don't need to supply with <b><code> na.rm = TRUE </code></b>

In [ ]:
by_tailnum <- group_by(flights_sqlite, tailnum)

In [ ]:
delay <- summarise(by_tailnum,
                  count = n(),
                  dist = mean(distance),
                   delay = mean(arr_delay)
                )

In [ ]:
delay <- filter(delay, count > 20, dist < 2000)

In [ ]:
delay_local <- collect(delay)

<h3> Other Databases </h3>
<br/> PostgreSQL

In [ ]:
con = dbConnect(pg, user="", password="", host="localhost", port=5432, dbname="")

In [ ]:
flights_postgres <- tbl(src_postgres("nycflights13"), "flights")

In [ ]:
daily <- group_by(flights_postgres, year, month, day)

In [ ]:
bestwork <- daily %>%
    select(flight, arr_delay) %>%
    filter(arr_delay == min(arr_delay) || arr_delay == max(arr_delay))
bestwork$query

In [ ]:
ranked <- daily %>%
    select(arr_delay) %>%
    mutate(rank = rank(desc(arr_delay))
ranked$query